In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git transformers accelerate xformers==0.0.16 wandb


In [24]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.16 requires torch==1.13.1, but you have torch 2.0.1+cu118 which is incompatible.


In [25]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .

Cloning into 'diffusers'...
remote: Enumerating objects: 28438, done.
remote: Counting objects: 100% (2434/2434), done.
remote: Compressing objects: 100% (1050/1050), done.
remote: Total 28438 (delta 1679), reused 1840 (delta 1208), pack-reused 26004
Receiving objects: 100% (28438/28438), 33.16 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (20554/20554), done.
/content/diffusers/examples/controlnet/diffusers/examples/controlnet/diffusers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/diffusers/examples/controlnet/diffusers/examples/controlnet/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-0.editable-py3-none-any.whl siz

In [26]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
%cd examples/controlnet

/content/diffusers/examples/controlnet/diffusers/examples/controlnet/diffusers/examples/controlnet


In [28]:
!ls
!pip install -r requirements.txt

README.md	       requirements.txt		 train_controlnet.py
requirements_flax.txt  train_controlnet_flax.py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
!wget https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/0/conditioning_image/image.jpg
!mv image.jpg image0.jpg
!wget https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/1/conditioning_image/image.jpg
!mv image.jpg image1.jpg

--2023-05-29 14:56:45--  https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/0/conditioning_image/image.jpg
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 34.200.186.24, 23.22.30.163, 3.216.183.114, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|34.200.186.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21234 (21K) [image/jpeg]
Saving to: ‘image.jpg’

image.jpg           100%[===================>]  20.74K  --.-KB/s    in 0.06s   

2023-05-29 14:56:45 (330 KB/s) - ‘image.jpg’ saved [21234/21234]

--2023-05-29 14:56:45--  https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/1/conditioning_image/image.jpg
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 34.200.186.24, 23.22.30.163, 3.216.183.114, ...
Connecting to datasets-s

In [30]:
!mkdir model

In [31]:
!accelerate config default

# Then move to your "home" dir:

!mv /root/.cache/huggingface/accelerate/default_config.yaml /content/

# Then edit manually and put back:

!mv /content/default_config.yaml /root/.cache/huggingface/accelerate/

# And update (?) (Possibly this is not necessary)

!accelerate config update

2023-05-29 14:21:55.826570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
2023-05-29 14:22:08.132656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Sucessfully updated the configuration file at /root/.cache/huggingface/accelerate/default_config.yaml.


In [39]:
%env MODEL_DIR=runwayml/stable-diffusion-v1-5
%env OUTPUT_DIR=model/

!accelerate launch train_controlnet.py \
 --pretrained_model_name_or_path=$MODEL_DIR \
 --controlnet_model_name_or_path=lllyasviel/sd-controlnet-seg \
 --dataset_name=baptistecolle/sam-controlnet-final \
 --resolution=512 \
 --learning_rate=1e-5 \
 --validation_image "./image0.png" "./image1.png" \
 --validation_prompt "A woman wearing a net on her head cutting a cake. " "A young boy standing in front of a computer keyboard." \
 --train_batch_size=4 \
 --image_column=image \
 --caption_column=text \
 --report_to=wandb

env: MODEL_DIR=runwayml/stable-diffusion-v1-5
env: OUTPUT_DIR=model/
2023-05-29 14:58:17.116679: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-29 14:58:25.674680: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 1.13.1+cu117 with CUDA 1107 (you have 2.0.1+cu118)
    Python  3.10.9 (you have 3.10.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:258: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
05/29/2023 14:58:31 - INFO - __main__ - Distributed environment: 

In [34]:
!dmesg | egrep -i -B100 'killed process'

[   11.647059] LoadPin: kernel-module pinning-excluded obj="/lib/modules/5.15.107+/kernel/net/sched/sch_htb.ko" pid=737 cmdline="/sbin/modprobe -q -- sch_htb"
[   11.666444] HTB: quantum of class 10001 is big. Consider r2q change.
[   11.678454] LoadPin: kernel-module pinning-excluded obj="/lib/modules/5.15.107+/kernel/net/sched/cls_u32.ko" pid=741 cmdline="/sbin/modprobe -q -- cls_u32"
[   11.694513] u32 classifier
[   11.698256]     input device check on
[   11.702047]     Actions configured
[   12.256653] loop0: detected capacity change from 0 to 230686720
[   12.286335] EXT4-fs (loop0): mounted filesystem with ordered data mode. Opts: (null). Quota mode: none.
[   12.622368] LoadPin: kernel-module pinning-excluded obj="/lib/modules/5.15.107+/kernel/net/netfilter/xt_nat.ko" pid=824 cmdline="/sbin/modprobe -q -- ipt_DNAT"
[   13.476546] LoadPin: kernel-module pinning-excluded obj="/lib/modules/5.15.107+/kernel/drivers/net/veth.ko" pid=966 cmdline="/sbin/modprobe -q -- rtnl-link-veth"